In [3]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install rapidfuzz

In [13]:
import pandas as pd  
from rapidfuzz import fuzz  
import re 
from tqdm import tqdm
import json  

def preprocess_company_name(name):
    """
    Preprocesses a company name by removing punctuation, converting to lowercase,
    and removing common words.
    """
    name = str(name)  # Convert the name to a string
    punctuation = r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]'  # Define a regular expression pattern for punctuation
    remove_words = ['ltd', 'inc', 'co', 'llc', 'of', 'medical', 'technology', 'research', 'corp', "lda", 'technologies',
                    'gmbh', 'and', 'technologies', 'the', 'corporation', 'company', 'incorporated', 'plc', 'sa', 'spa',
                    'ag', 'limited', 'group', 'srl', 'sarl', 'nv', 'spa', 'asa', 'gmbh', 'ab', 'aps', 'pty', 'eurl',
                    'aps', 'sas', "bv", "lp", "kg", "pvt", "pharma", "pharmacy", "pharmaceuticals", "pharmaceutials",
                    "pharmaceutical", "biopharma", "biopharmaceutical", "biopharmaceuticals", "therapeutics",
                    "biotherapeutics", "hospital", "biosciences", "biotech", "bioscience", "biotechnology",
                    "chemicals", "chemical", "biology", "sciences", "science", "biotechnologies", "laboratories",
                    "innovations", "diagnostics"]  # Define a list of common words to remove from the name

    name = re.sub(punctuation, '', name)  # Remove punctuation from the name
    name = name.lower().strip()  # Convert the name to lowercase and remove spaces
    words = name.split()  # Split the name into individual words
    words = [word for word in words if word not in remove_words]  # Remove common words from the list of words
    return ' '.join(words)  # Join the processed words back into a string


def find_similar_companies(companies, key_list, threshold=95):
    """
    Finds similar companies based on fuzzy string matching.
    """
    filtered_key_list = [preprocess_company_name(name) for name in key_list]  # Preprocess the key_list
    org_dict = {key: [] for key in key_list}  # Create an empty dictionary to store the similar companies

    for company in tqdm(companies):  # Iterate over each company name
        highest_similarity = 0  # Initialize the highest similarity score
        filtered_company = preprocess_company_name(company)  # Preprocess the current company name
        similar_company = ''  # Initialize the similar company name

        for i, key in enumerate(key_list):  # Iterate over the key list
            score = fuzz.ratio(filtered_company, filtered_key_list[i])  # Calculate the similarity score
            if score > highest_similarity:  # Check if the current score is higher than the highest similarity score
                similar_company = key  # Update the similar company name
                highest_similarity = score  # Update the highest similarity score

        if highest_similarity < threshold:  # Check if the highest similarity score is below the threshold
            key_list.append(company)  # Add the current company to the key list
            filtered_key_list.append(filtered_company)  # Add the preprocessed company name to the filtered key list
            org_dict[company] = []  # Create an empty list for the current company
        else:
            org_dict[similar_company].append(company)  # Add the current company to the list of similar companies

    return org_dict  # Return the dictionary of similar companies

In [14]:
def main():
    # Read and merge the company datasets
    df1 = pd.read_csv('bmt_company.csv').rename(columns={'bmt_company_id': 'Company_Id'})  
    df2 = pd.read_csv('cortellis_company.csv').rename(columns={'cortellis_company_id': 'Company_Id'})  
    df3 = pd.read_csv('pp_company.csv').rename(columns={'pp_company_id': 'Company_Id'}) 
    unique_companies_df = pd.concat([df1, df2, df3], ignore_index=True)  # Merge the datasets

    # Get the unique company names
    companies = list(set(unique_companies_df['company_nm']))  # Extract the unique company names

    # Initialize key_list and find similar companies
    key_list = []  # Create an empty key list
    org_dict = find_similar_companies(companies, key_list)  # Find similar companies

    # Save the dictionary to a JSON file
    with open("org_dict__Threshold.json", 'w') as f:  # Open the JSON file for writing
        json.dump(org_dict, f)  # Write the organization dictionary to the file

    # Create a DataFrame with entity, matched entities, and count
    df = pd.DataFrame()
    df['entity'] = org_dict.keys()
    df['matched_entities'] = org_dict.values()
    df['matched_entities_count'] = [len(i) for i in org_dict.values()]  # Create a DataFrame with the organization dictionary values
    df= df.sort_values(by = 'matched_entities_count', ascending = False)
    df = df.to_excel('organization_mapping.xlsx',index = False)


if __name__ == '__main__':
    main()  # Execute the main function


100%|██████████| 100/100 [00:00<00:00, 40302.72it/s]

None
